In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import linkage, dendrogram
from sklearn.cluster import AgglomerativeClustering
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
df=pd.read_csv('../input/mall-customers/Mall_Customers.csv')

In [ ]:
df.head()

In [ ]:
df.dtypes

In [ ]:
df=df.rename(columns={'Genre':'Gender'})

In [ ]:
df.dtypes

In [ ]:
df=pd.get_dummies(data=df,columns=['Gender'])

In [ ]:
df=df.drop('Gender_Male',axis=1)

In [ ]:
df.head()

In [ ]:
df.isnull().sum()

We will drop the customer ID column as it is not useful for our analysis.

In [ ]:
df=df.drop('CustomerID',axis=1)

In [ ]:
sns.scatterplot(x='Age',y='Spending Score (1-100)',data=df)




We observe that spending score is much higher for people between the age group 20 and 40.

In [ ]:
sns.scatterplot(x='Annual Income (k$)',y='Spending Score (1-100)',data=df)

We observe that there is a definite pattern for spending score for the annual income range between 40k and 60k.

In [ ]:
sns.scatterplot(x='Gender_Female',y='Spending Score (1-100)',data=df)


We observe that females have more spending score when compared to that of the males.

In [ ]:
sns.distplot(df['Spending Score (1-100)'])

In [ ]:
sns.boxplot(df['Spending Score (1-100)'])

We observe that the spending score is normally distributed from both the box plot and the distplot.

In [ ]:
corr=df.corr()
ax=sns.heatmap(corr,annot=True)


From the heat map we observe that spending score is affected most by the age. It is also affected by the gender.
Annual inome is not that correlated with the spending score.


We will no create bins for the spending score and proceed with our analysis.

In [ ]:
def spending_bin(a):
    if(a in range(1,10)):
        return '1'
    elif(a in range(10,20)):
        return '2'
    elif(a in range(20,30)):
        return '3'
    elif(a in range(30,40)):
        return '4'
    elif(a in range(40,50)):
        return '5'
    elif (a in range(50,60)):
        return '6'
    elif (a in range(60,70)):
        return '7'
    elif (a in range(70,80)):
        return '8'
    elif (a in range(80,90)):
        return '9'
    
list5=list(map(spending_bin,df['Spending Score (1-100)']))
df['Spending_Score_bin']=list5
df.head()

In [ ]:
sns.countplot(df['Spending_Score_bin'])

Majority of the people have spending score in the range of 40 to 50.

In [ ]:
# Creating seperate dataframes for male and female
df_f=df[df['Gender_Female']==1]
df_m=df[df['Gender_Female']==0]

In [ ]:
sns.countplot(df_f['Spending_Score_bin'])

In [ ]:
df_female=df_f['Spending_Score_bin'].value_counts().sort_index(ascending=False)

In [ ]:
sns.countplot(df_m['Spending_Score_bin'])

In [ ]:
df_male=df_m['Spending_Score_bin'].value_counts().sort_index(ascending=False)

In [ ]:
df_female

In [ ]:
df_male





From the bar plot and the value counts we observe that the number of females in the higher range of spending score are more when compared to males.
Also under most of the categories the female population tend to have higher spending score in comparison with that of the males.
Men tend to have the maximum spending score in the range of 40 to 60.
Both male and female have their maximum score in the range of 40 to 60. In order to push the spending score even more we can give more rewards/ cashback points to that category of people.

In [ ]:
sns.barplot(x=df_m['Spending_Score_bin'].sort_values(),y=df_m['Annual Income (k$)'])

Male population having the highest salary range have a spending score in the range of 10 to 20.

In [ ]:
sns.barplot(x=df_f['Spending_Score_bin'].sort_values(),y=df_f['Annual Income (k$)'])

Female population having the highest salary range have a spending score in the range of 20 to 30.

In [ ]:
df=df.drop(['Spending_Score_bin'],axis=1)

We need not scale our data as our data is unitless. We will perform Hierarchical clustering now.


**HIERARCHICAL CLUSTERING**

In [ ]:
plt.figure(figsize=[10,10])
merg = linkage(df, method='ward')
dendrogram(merg, leaf_rotation=90)
plt.title('Dendrogram')
plt.xlabel('Data Points')
plt.ylabel('Euclidean Distances')
plt.show()

In [ ]:
hie_clus = AgglomerativeClustering(n_clusters=6, affinity='euclidean', linkage='ward')
cluster2 = hie_clus.fit_predict(df)

df_h = df.copy(deep=True)
df_h['label'] = cluster2

In [ ]:
sns.pairplot(df_h,hue='label',palette='husl')

Category 1 and 5 are almost similar. We can consider them as a single group. So we effectively have 5 groups.
We will design a model using the same.


In [ ]:
hie_clus = AgglomerativeClustering(n_clusters=5, affinity='euclidean', linkage='ward')
cluster2 = hie_clus.fit_predict(df)

df_h = df.copy(deep=True)
df_h['label'] = cluster2

In [ ]:
sns.pairplot(df_h,hue='label',palette='husl')

The five categories observed based on income and spending score are as follows:
*     0-Medium income and Medium spending score.
*     1-High income and low spending score.
*     2-High income and high spending score.
*     3-Low income and high spending score.
*     4-Low income and low spending score.